In [2]:
import os
import requests
from urllib.request import urlopen
import pathlib
from bs4 import BeautifulSoup
from datetime import date
import datetime as dt
import pandas as pd
import sys
import csv
import numpy as np
import multiprocessing
import re
from collections import namedtuple
# For pausing
from time import sleep
os.environ["JAVA_HOME"] = "/Library/Java/JavaVirtualMachines/jdk1.8.0_251.jdk/Contents/Home"
import pickle
from itertools import repeat
import psycopg2
import pandas.io.sql as sqlio
# adding Folder_2/subfolder to the system path
sys.path.insert(0, '/Volumes/TOSHIBA_EXT/ISB/clinical_trials/pymetamap-master')
# sys.path.insert(0, '/Users/Kamileh/Work/ISB/NCATS_BiomedicalTranslator/Projects/ClinicalTrials/ETL_Python')

from pymetamap import MetaMap
import random
from run_metamap import metamapper

import json
# import argparse
# import lxml

# from Authentication import *
import requests
# from requests.adapters import HTTPAdapter
# from requests.packages.urllib3.util.retry import Retry

import urllib
import zipfile

In [ ]:
# # Setup UMLS Server global vars
# # metamap_base_dir = '/Volumes/TOSHIBA_EXT/ISB/clinical_trials/public_mm/' # for running on local
# # metamap_base_dir = "/users/knarsinh/projects/clinical_trials/metamap/public_mm/"
# # metamap_bin_dir = 'bin/metamap18' # uncomment for running on local
# metamap_base_dir = "{}/metamap/".format(pathlib.Path.cwd().parents[0])
# metamap_bin_dir = 'bin/metamap20'
# metamap_pos_server_dir = 'bin/skrmedpostctl'
# metamap_wsd_server_dir = 'bin/wsdserverctl'


In [3]:
# Setup UMLS Server global vars

global metamap_pos_server_dir
global metamap_wsd_server_dir
metamap_pos_server_dir = 'bin/skrmedpostctl'
metamap_wsd_server_dir = 'bin/wsdserverctl'
vocab_restriction = ['SNOMEDCT_US', 'SNOMEDCT_VET', 'ICD10CM', 'ICD10CM', 'ICD10PCS', 'ICD9CM', 'ICD9CM']



In [19]:
def check_os():
    if "linux" in sys.platform:
        print("Linux platform detected")
        metamap_base_dir = "{}/metamap/".format(pathlib.Path.cwd().parents[0])
        metamap_bin_dir = 'bin/metamap20'
    else:
        metamap_base_dir = '/Volumes/TOSHIBA_EXT/ISB/clinical_trials/public_mm/' # for running on local
        metamap_bin_dir = 'bin/metamap18'
        
    return {"metamap_base_dir":metamap_base_dir, "metamap_bin_dir":metamap_bin_dir}
        
metamap_dirs = check_os()
metamap_dirs['metamap_bin_dir']

# /Volumes/TOSHIBA_EXT/ISB/clinical_trials/public_mm

'bin/metamap18'

In [5]:
def get_raw_ct_data():
    term_program_flag = True
    global data_dir
    global data_extracted
    
    # get all the links and associated dates of upload into a dict called date_link
    url_all = "https://aact.ctti-clinicaltrials.org/pipe_files"
    response = requests.get(url_all)
    soup = BeautifulSoup(response.text)
    body = soup.find_all('option') #Find all
    date_link = {}
    for el in body:
        tags = el.find('a')
        try:

            zip_name = tags.contents[0].split()[0]
            date = zip_name.split("_")[0]
            date = dt.datetime.strptime(date, '%Y%m%d').date()
            date_link[date] = tags.get('href')
        except:
            pass
    # get the date of the latest upload
    latest_file_date = max(date_link.keys())
    
     # get the corresponding download link of the latest upload so we can download the raw data
    url = date_link[latest_file_date]
    data_dir = "{}/data".format(pathlib.Path.cwd().parents[0])
    data_extracted = data_dir + "/{}_extracted".format(latest_file_date)
    data_path = pathlib.Path.cwd().parents[0] / "{}/{}_pipe-delimited-export.zip".format(data_dir, latest_file_date)

    # if folder containing most recent data doesn't exist, download and extract it into data folder
    if not os.path.exists(data_path):
        # flag below for terminating program if latest download exists (KG is assumed up to date)
        term_program_flag = False  
        print("Downloading Clinical Trial data as of {}".format(latest_file_date))
        req = requests.get(url)
        with open(data_path, 'wb') as download:
            download.write(req.content)
        print("Finished download")
        # extract the downloaded zip file
        with zipfile.ZipFile(data_path, 'r') as download:
            print("Unzipping data")
            download.extractall(data_extracted) 
        print("Finished extracting zip")
    else:
        print("KG is already up to date.")
    
    return {"term_program_flag": term_program_flag, "data_extracted_path": data_extracted}
    
    

flag_and_path = get_raw_ct_data()


Finished download
Unzipping data
Finished extracting zip


In [6]:
def merge_raw_ct_data(data_extracted):
    
    # read in pipe-delimited files
    conditions_df = pd.read_csv(data_extracted + '/conditions.txt', sep='|', index_col=False, header=0)
    interventions_df = pd.read_csv(data_extracted + '/interventions.txt', sep='|', index_col=False, header=0)
    browse_conditions_df = pd.read_csv(data_extracted + '/browse_conditions.txt', sep='|', index_col=False, header=0)
    browse_interventions_df = pd.read_csv(data_extracted + '/browse_interventions.txt', sep='|', index_col=False, header=0)

    # rename and drop df relevant columns to prepare for merging
    interventions_df = interventions_df.rename(columns={'id': 'int_id',
                                                        'nct_id': 'int_nctid',
                                                        'intervention_type': 'int_type',
                                                        'name': 'int_name',
                                                        'description': 'int_description'})
    interventions_df = interventions_df.drop(columns=['int_id', 'int_description'])
    conditions_df = conditions_df.rename(columns={'id': 'con_id',
                                                  'nct_id': 'con_nctid',
                                                  'name': 'con_name',
                                                  'downcase_name': 'con_downcase_name'})
    conditions_df = conditions_df.drop(columns=['con_id', 'con_name'])
    browse_interventions_df = browse_interventions_df.rename(columns={'id': 'browseint_id',
                                                                      'nct_id': 'browseint_nctid',
                                                                      'mesh_term': 'browseint_meshterm',
                                                                      'downcase_mesh_term': 'browseint_meshterm_downcase',
                                                                      'mesh_type': 'browseint_meshtype'})

    browse_interventions_df = browse_interventions_df.drop(columns=['browseint_id', 'browseint_meshterm'])
    browse_conditions_df = browse_conditions_df.rename(columns={'id': 'browsecon_id',
                                                                'nct_id': 'browsecon_nctid',
                                                                'mesh_term': 'browsecon_meshterm',
                                                                'downcase_mesh_term': 'browsecon_meshterm_downcase',
                                                                'mesh_type': 'browsecon_meshtype'})
    browse_conditions_df = browse_conditions_df.drop(columns=['browsecon_id', 'browsecon_meshterm'])                                                                                                                          

    # merge conditions_df and interventions_df since they have relevant terms 
    df = pd.merge(conditions_df, interventions_df, left_on='con_nctid', right_on = 'int_nctid')
    df_dedup = df.drop_duplicates(subset = ['con_downcase_name', 'int_name', 'con_nctid'],
                                          keep = 'first').reset_index(drop = True)
    return df_dedup

ct_data = merge_raw_ct_data(flag_and_path.get("data_extracted_path"))


In [7]:
def preprocess_ct_data(ct_data):    
    # obtain relevant columns
    ct_extract = pd.DataFrame(ct_data[['con_nctid', 'con_downcase_name', 'int_type', 'int_name']])
    ct_extract = ct_extract.rename(columns={'con_nctid': 'nctid'})
    # get CURIE column for nct_id column (https://bioregistry.io/registry/clinicaltrials)

    ct_extract['nctid_curie'] = ct_extract['nctid']
    ct_extract['nctid_curie'] = 'clinicaltrials:' + ct_extract['nctid'].astype(str)  # generate CURIEs for each clinical trials study

    ct_final = pd.DataFrame(columns=['subject','predicate','object', 'subject_name','object_name','category'])

    ct_final['subject'] = 'condition:' + ct_extract['con_downcase_name'].astype(str)
    ct_final['predicate'] = 'biolink:related_to'
    ct_final['object'] = 'intervention:' + ct_extract['int_name'].astype(str)   # this will not all be RxNorm CURIEs since some interventions are not drugs
    ct_final.subject_name = ct_extract.con_downcase_name
    ct_final.object_name = ct_extract.int_name
    ct_final.category = 'biolink:Association'
    ct_final['nctid'] = ct_extract['nctid']
    ct_final['nctid_curie'] = ct_extract['nctid_curie']

    return(ct_final)


ct_final = preprocess_ct_data(ct_data)
ct_final


,subject,predicate,object,subject_name,object_name,category,nctid,nctid_curie
0,condition:age-related macular degeneration,biolink:related_to,intervention:Conjunctival and nasopharyngeal b...,age-related macular degeneration,Conjunctival and nasopharyngeal bacteria cultures,biolink:Association,NCT00831961,clinicaltrials:NCT00831961
1,condition:mastocytosis,biolink:related_to,intervention:masitinib (AB1010),mastocytosis,masitinib (AB1010),biolink:Association,NCT00831974,clinicaltrials:NCT00831974
2,condition:haemophilia a with inhibitors,biolink:related_to,intervention:eptacog alfa (activated),haemophilia a with inhibitors,eptacog alfa (activated),biolink:Association,NCT01561924,clinicaltrials:NCT01561924
3,condition:haemophilia b with inhibitors,biolink:related_to,intervention:eptacog alfa (activated),haemophilia b with inhibitors,eptacog alfa (activated),biolink:Association,NCT01561924,clinicaltrials:NCT01561924
4,condition:congenital bleeding disorder,biolink:related_to,intervention:eptacog alfa (activated),congenital bleeding disorder,eptacog alfa (activated),biolink:Association,NCT01561924,clinicaltrials:NCT01561924
...,...,...,...,...,...,...,...,...
1303041,condition:postoperative pain,biolink:related_to,intervention:LWI group,postoperative pain,LWI group,biolink:Association,NCT02284620,clinicaltrials:NCT02284620
1303042,condition:total knee arthroplasty,biolink:related_to,intervention:CFNB group,total knee arthroplasty,CFNB group,biolink:Association,NCT02284620,clinicaltrials:NCT02284620
1303043,condition:total knee arthroplasty,biolink:related_to,intervention:LWI group,total knee arthroplasty,LWI group,biolink:Association,NCT02284620,clinicaltrials:NCT02284620
1303044,condition:cystoid macular edema,biolink:related_to,"intervention:Ranibizumab,",cystoid macular edema,"Ranibizumab,",biolink:Association,NCT02294656,clinicaltrials:NCT02294656


In [8]:
def de_ascii_er(text):
    non_ascii = "[^\x00-\x7F]"
    pattern = re.compile(r"[^\x00-\x7F]")
    non_ascii_text = re.sub(pattern, ' ', text)
    return non_ascii_text

In [9]:
def start_metamap_servers(metamap_dirs):
    # Start servers
    os.system(metamap_dirs['metamap_base_dir'] + metamap_pos_server_dir + ' start') # Part of speech tagger
    os.system(metamap_dirs['metamap_base_dir'] + metamap_wsd_server_dir + ' start') # Word sense disambiguation 
    # # Sleep a bit to give time for these servers to start up
    sleep(60)

def stop_metamap_servers(metamap_dirs):
    # Stop servers
    os.system(metamap_dirs['metamap_base_dir'] + metamap_pos_server_dir + ' stop') # Part of speech tagger
    os.system(metamap_dirs['metamap_base_dir'] + metamap_wsd_server_dir + ' stop') # Word sense disambiguation 
        

In [10]:
start_metamap_servers(metamap_dirs)


Starting skrmedpostctl: 
started.
Starting wsdserverctl: 
started.
loading properties file /Volumes/TOSHIBA_EXT/ISB/clinical_trials/public_mm/WSD_Server/config/disambServer.cfg
WSD Server initializing disambiguation methods.
WSD Server databases and disambiguation methods have been initialized.
Could not listen on port : 5554 : Address already in use


In [ ]:
# def run_metamap(input_term, source_restriction, metamap_dirs):

#     mm = MetaMap.get_instance(metamap_dirs['metamap_base_dir'] + metamap_dirs['metamap_bin_dir'])
# #     
#     concepts_dict = dict()
#     if all(x is None for x in source_restriction):
#         try:
#             concepts,error = mm.extract_concepts([input_term],
#                                                  word_sense_disambiguation = True,
#                                                  prune = 10,
#                                                  composite_phrase = 1)
#             concepts_dict[input_term] = concepts
#         except:
#             concepts_dict[input_term] = None 
#     else:
#         try:
#             concepts,error = mm.extract_concepts([input_term],
#                                                  word_sense_disambiguation = True,
#                                                  restrict_to_sources=source_restriction,
#                                                  prune = 10,
#                                                  composite_phrase = 1)
#             concepts_dict[input_term] = concepts
#         except:
#             concepts_dict[input_term] = None 
#     return(concepts_dict)




In [ ]:
stop_metamap_servers(metamap_dirs)

In [ ]:
# def start_metamap_servers():
#     # Start servers
#     os.system(metamap_dirs['metamap_base_dir'] + metamap_pos_server_dir + ' start') # Part of speech tagger
#     os.system(metamap_dirs['metamap_base_dir'] + metamap_wsd_server_dir + ' start') # Word sense disambiguation 
#     # # Sleep a bit to give time for these servers to start up
#     sleep(60)

# def stop_metamap_servers():
#     # Stop servers
#     os.system(metamap_dirs['metamap_base_dir'] + metamap_pos_server_dir + ' stop') # Part of speech tagger
#     os.system(metamap_dirs['metamap_base_dir'] + metamap_wsd_server_dir + ' stop') # Word sense disambiguation 
        
# start_metamap_servers()    
# mm = MetaMap.get_instance(metamap_dirs['metamap_base_dir'] + metamap_dirs['metamap_bin_dir'])
# concepts_dict = dict()
# if all(x is None for x in source_restriction):
#     try:
#         concepts,error = mm.extract_concepts([input_term],
#                                                  word_sense_disambiguation = True,
#                                                  prune = 10,
#                                                  composite_phrase = 1)
#             concepts_dict[input_term] = concepts
#         except:
#             concepts_dict[input_term] = None 
#     else:
#         try:
#             concepts,error = mm.extract_concepts([input_term],
#                                                  word_sense_disambiguation = True,
#                                                  restrict_to_sources=source_restriction,
#                                                  prune = 10,
#                                                  composite_phrase = 1)
#             concepts_dict[input_term] = concepts
#         except:
#             concepts_dict[input_term] = None 
#     stop_metamap_servers()
#     return(concepts_dict)


In [11]:
def preprocess_cols(ct_final, metamap_dirs):
    # prep a list to hold dicts of de-asciied cols from the ClinTrials df
    transformations = {}

    # get unique lists of the concepts to MetaMap from the ClinTrials df
    conditions = list(ct_final['subject_name'].unique())
    conditions = [x for x in conditions if pd.isnull(x) == False]
    interventions = list(ct_final['object_name'].unique())
    interventions = [x for x in interventions if pd.isnull(x) == False]
    
    # report the number of unique conditions and interventions to get idea of KG at early stage
    print("# of unique CONDITIONS present in Clinical Trials dataset: {}".format(len(conditions)))
    print("# of unique INTERVENTIONS present in Clinical Trials dataset: {}".format(len(interventions)))
    
    # only remove non-ASCII for MetaMap versions prior to 2020 release
    # if stmnt is checking if 20 for 2020 ---> bin/metamap20 is MetaMap release 2020
    metamap_version = [int(s) for s in re.findall(r'\d+', metamap_dirs.get('metamap_bin_dir'))]
    if metamap_version[0] >= 20:
        print("MetaMap version >= 2020, will not remove ")
        conditions_unchanged = dict((i, i) for i in conditions)
        interventions_unchanged = dict((i, i) for i in interventions) 
        transformations["conditions"] = conditions_unchanged
        transformations["interventions"] = interventions_unchanged      
    else:
        print("Removing ASCII char from cols.")
        print("This step is unnecessary for MetaMap 2020+")
        conditions_translated = dict((i, de_ascii_er(i)) for i in conditions)
        interventions_translated = dict((i, de_ascii_er(i)) for i in interventions)   
        
        transformations["conditions"] = conditions_translated
        transformations["interventions"] = interventions_translated  
        
    return(transformations)
        

transformed = preprocess_cols(ct_final, metamap_dirs)


# of unique CONDITIONS present in Clinical Trials dataset: 89542
# of unique INTERVENTIONS present in Clinical Trials dataset: 369500
Removing ASCII char from cols.
This step is unnecessary for MetaMap 2020+


In [ ]:
#     con_test = dict(random.sample(list(transformations.get("conditions").items()), 10))  
#     int_test = dict(random.sample(list(transformations.get("interventions").items()), 10))
    
    # MAP CONDITIONS AND INTERVENTIONS TO METAMAP
    
        

#     metamapped_conditions = multiprocessing.Pool(num_workers).starmap(run_metamap, zip(list(con_test.values()),
#                                                                                           [vocab_restriction]*len(list(con_test.values()))))
#     with multiprocessing.Pool(processes=3) as pool:
#         results = pool.starmap(merge_names, product(names, repeat=2))

        

    
    
#     metamapped_conditions = multiprocessing.Pool(multiprocessing.cpu_count() - 1).starmap(run_metamap,
#         zip(list(transformations['conditions'].values()),
#             [vocab_restriction]*len(list(transformations['conditions'].values())), metamap_dirs))
#     metamapped_interventions = multiprocessing.Pool(multiprocessing.cpu_count() - 1).starmap(run_metamap, 
#                                                                                              zip(list(transformations['interventions'].values()),
#                                                                                                  [[None]]*len(list(transformations['conditions'].values()))))
    
#     metamapped_conditions = multiprocessing.Pool(multiprocessing.cpu_count() - 1).starmap(run_metamap,
#         zip(list(con_test),
#             [vocab_restriction]*len(list(transformations['conditions'].values())), metamap_dirs))
#     metamapped_interventions = multiprocessing.Pool(multiprocessing.cpu_count() - 1).starmap(run_metamap, 
#                                                                                              zip(list(int_test),
#                                                                                                  [[None]]*len(list(transformations['conditions'].values()))))

    
    # only remove non-ASCII for MetaMap versions prior to 2020 release
#     metamap_dir = "{}/metamap".format(pathlib.Path.cwd().parents[0]) # uncomment for  server edition
#     metamap_dir = "/Volumes/TOSHIBA_EXT/ISB/clinical_trials"
#     print(list(pathlib.Path(metamap_base_dir).glob('*.tar.bz2'))) # uncomment for use on server
#     print(list(pathlib.Path(metamap_dir).glob('*.tar.bz2')))   # uncomment for use on local
#     print(metamap_base_dir)

    


#     if not all(condition.isascii() for condition in conditions):
#         print("Non-ASCII chars are detected in Conditions col from ClinTrial (not checking other cols), proceed with text processing")
#         print("This step is unnecessary for MetaMap 2020+")
#         # process lists to remove non-ascii chars (this is not required for MetaMap 2020!!!!)
#         conditions_translated = dict((i, preprocess_cols_for_metamap(i)) for i in conditions)
#         interventions_translated = dict((i, preprocess_cols_for_metamap(i)) for i in interventions)
        
#         metamap_compatible["conditions"] = conditions_translated
#         metamap_compatible["interventions"] = interventions_translated

#     else:
#         print("No non-ASCII chars detected or they are present, but we're not using MetaMap versions prior to 2020, no text processing required")
#         metamap_compatible["conditions"] = conditions
#         metamap_compatible["interventions"] = interventions

#     return(metamap_compatible)

          

# transformations = cols_transformations(ct_final, metamap_dirs)

In [ ]:
metamapped_conditions

In [ ]:
transformations

In [24]:
metamap_dirs

{'metamap_base_dir': '/Volumes/TOSHIBA_EXT/ISB/clinical_trials/public_mm/',
 'metamap_bin_dir': 'bin/metamap18'}

In [12]:
type(transformed)

dict

In [27]:
if __name__ == '__main__':
#     metamap_dirs = check_os()
#     flag_and_path = get_raw_ct_data()
#     ct_data = merge_raw_ct_data(flag_and_path.get("data_extracted_path"))
#     ct_final = preprocess_ct_data(ct_data)
    start_metamap_servers(metamap_dirs)
    transformed = preprocess_cols(ct_final, metamap_dirs)
    con_test = dict(random.sample(list(transformed.get("conditions").items()), 10))  
    int_test = dict(random.sample(list(transformed.get("interventions").items()), 10))
#     with multiprocessing.Pool(processes=multiprocessing.cpu_count() - 1) as p:
#         metamapped_conditions = p.starmap(metamapper, zip(list(con_test.values()), [vocab_restriction]*len(list(con_test.values())), [metamap_dirs]*len(list(con_test.values()))))
    with multiprocessing.Pool(processes=multiprocessing.cpu_count() - 1) as p:
        metamapped_conditions = p.starmap(metamapper, zip(list(con_test.values()), [vocab_restriction]*len(list(con_test.values())), [metamap_dirs]*len(list(con_test.values()))))
    stop_metamap_servers(metamap_dirs)
    

Starting skrmedpostctl: 
started.
Starting wsdserverctl: 
started.
loading properties file /Volumes/TOSHIBA_EXT/ISB/clinical_trials/public_mm/WSD_Server/config/disambServer.cfg
WSD Server initializing disambiguation methods.
WSD Server databases and disambiguation methods have been initialized.
Could not listen on port : 5554 : Address already in use
# of unique CONDITIONS present in Clinical Trials dataset: 89542
# of unique INTERVENTIONS present in Clinical Trials dataset: 369500
Removing ASCII char from cols.
This step is unnecessary for MetaMap 2020+
Stopping skrmedpostctl: 
Stopping Tagger Server process..
Process 52569 stopped
Stopping wsdserverctl: 
Stopping WSD Server process..
Process 52571 stopped


/Volumes/TOSHIBA_EXT/ISB/clinical_trials/public_mm/bin/skrmedpostctl: line 50: kill: (52569) - No such process
/Volumes/TOSHIBA_EXT/ISB/clinical_trials/public_mm/bin/wsdserverctl: line 55: kill: (52571) - No such process


In [26]:
stop_metamap_servers(metamap_dirs)

Stopping skrmedpostctl: 
Server not running
Stopping wsdserverctl: 
Server not running


In [ ]:
# def driver():
#     metamap_dirs = check_os()
#     flag_and_path = get_raw_ct_data()
#     ct_data = merge_raw_ct_data(flag_and_path.get("data_extracted_path"))
#     ct_final = preprocess_ct_data(ct_data)
#     start_metamap_servers(metamap_dirs)
#     metamap_compatible_cols = cols_transformations(ct_final, metamap_dirs)
#     stop_metamap_servers(metamap_dirs)

# driver()        

In [ ]:
def driver():
    metamap_dirs = check_os()
    flag_and_path = get_raw_ct_data()
    
    if flag_and_path.get("term_program_flag"):
        print("Exiting")
        quit()
    else:
        ct_data = merge_raw_ct_data(flag_and_path.get("data_extracted_path"))
        ct_final = preprocess_ct_data(ct_data)
#         metamap_compatible_cols = select_cols(ct_final)

driver()        

In [ ]:
rm.metamapper()